<a href="https://colab.research.google.com/github/mariborges22/Logistics-Pattern-Analysis-with-K-means-Public-Version/blob/master/Segmenta%C3%A7%C3%A3o_de_produtos_do_estoque_com_K_means.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd

df = pd.read_excel("/content/drive/MyDrive/BI_MOV_SUPORTE.xlsx", engine='calamine')
df.head(25)

In [ ]:
!pip install python-calamine

In [ ]:
# 2. Verificar duplicatas
num_duplicatas = df.duplicated().sum()
print(f"Duplicatas encontradas: {num_duplicatas}")

# 3. Remover espaços em branco
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# 4. Verificar células vazias (ou com apenas espaços)
celulas_vazias = df.applymap(lambda x: isinstance(x, str) and x.strip() == "").sum()
print("\nCélulas vazias ou com espaços por coluna:")
print(celulas_vazias)

# 5. Verificar valores únicos por coluna
print("\nValores únicos por coluna:")
print(df.nunique())

# 6. Informações gerais
print("\nResumo do DataFrame:")
print(df.info())


In [ ]:
# Remove rows where 'ano_movimentacao' is 2302
df = df[df['ano_movimentacao'] != 2302].copy()

# Verify the removal
print("Checking df for ano_movimentacao == 2302 after removal:")
display(df[df['ano_movimentacao'] == 2302])

In [ ]:
df_movement = df[['categoria_produto', 'subcategoria_produto', 'nomelocalestoque', 'filial', 'ano_movimentacao', 'qte_movimentacao', 'nomeempresa', 'tipo_movimentacao']].copy()
display(df_movement.head())

In [ ]:
df_movement['qte_movimentacao'] = pd.to_numeric(df_movement['qte_movimentacao'], errors='coerce')
df_aggregated_movement = df_movement.groupby(['categoria_produto', 'subcategoria_produto', 'nomelocalestoque', 'filial', 'ano_movimentacao', 'nomeempresa', 'tipo_movimentacao'])['qte_movimentacao'].sum().reset_index()
display(df_aggregated_movement.head())

In [ ]:
# Check for the year 2302 in the aggregated dataframe
print("Checking df_aggregated_movement for ano_movimentacao == 2302:")
display(df_aggregated_movement[df_aggregated_movement['ano_movimentacao'] == 2302])

# If found in aggregated data, check the original dataframe
print("\nChecking original df for ano_movimentacao == 2302:")
display(df[df['ano_movimentacao'] == 2302])

In [ ]:
df_sorted_movement = df_aggregated_movement.sort_values(by='tipo_movimentacao', ascending=False)
display(df_sorted_movement.head(10))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

df_sorted_movement['category_subcategory'] = df_sorted_movement['categoria_produto'] + ' - ' + df_sorted_movement['subcategoria_produto']

plt.figure(figsize=(12, 6))
sns.barplot(x='category_subcategory', y='qte_movimentacao', data=df_sorted_movement.head(10))
plt.xticks(rotation=45, ha='right')
plt.title('Top 10 Product Categories and Subcategories by Movement')
plt.xlabel('Category - Subcategory')
plt.ylabel('Total Movement Quantity')
plt.tight_layout()
plt.show()

In [ ]:
pivot_table = df_sorted_movement.head(10).pivot_table(index='categoria_produto', columns=['filial', 'nomelocalestoque', 'tipo_movimentacao'], values='qte_movimentacao', fill_value=0)

plt.figure(figsize=(14, 8))
sns.heatmap(pivot_table, annot=True, fmt=".1f", cmap="YlGnBu")
plt.title('Movement Patterns of Top Categories/Subcategories by Branch and Location')
plt.xlabel('Branch - Location')
plt.ylabel('Category - Subcategory')
plt.tight_layout()
plt.show()

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Select only numeric columns for scaling
df_numeric = df.select_dtypes(include=['number'])

scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(df_numeric)

In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

inertia = []
# Try a range of cluster numbers to find the optimal value using the elbow method
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, random_state=42, n_init=10)
    kmeans.fit(data_scaled)
    inertia.append(kmeans.inertia_)

# Plot the elbow curve
plt.plot(range(1, 11), inertia)
plt.title('Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')
plt.show()

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler

# Select numeric columns from the aggregated dataframe for clustering
df_aggregated_numeric = df_aggregated_movement.select_dtypes(include=['number'])

# Scale the numeric data in the aggregated dataframe
scaler_agg = MinMaxScaler()
data_aggregated_scaled = scaler_agg.fit_transform(df_aggregated_numeric)

# Apply KMeans clustering to the scaled aggregated data
kmeans = KMeans(n_clusters=6, random_state=42, n_init=10)
kmeans.fit(data_aggregated_scaled)

# Add the cluster labels to the aggregated dataframe
df_aggregated_movement['cluster'] = kmeans.labels_

In [ ]:
# Analyze cluster characteristics for categorical features

def display_cluster_characteristics(cluster_df, cluster_label):
    """Displays the characteristics of a given cluster."""
    print(f"\n--- Características do Cluster {cluster_label} ---")

    print("\nQuantidade Movimentada:")
    display(cluster_df['qte_movimentacao'].describe())

    print("\nCategorias de Produto mais frequentes:")
    display(cluster_df['categoria_produto'].value_counts().head())

    print("\nSubcategorias de Produto mais frequentes:")
    display(cluster_df['subcategoria_produto'].value_counts().head())

    print("\nFiliais mais frequentes:")
    display(cluster_df['filial'].value_counts().head())

    print("\nLocais de Estoque mais frequentes:")
    display(cluster_df['nomelocalestoque'].value_counts().head())

    print("\nAnos de Movimentação mais frequentes:")
    display(cluster_df['ano_movimentacao'].value_counts().head())

    print("\nEmpresas de Movimentação mais frequentes:")
    display(cluster_df['nomeempresa'].value_counts().head())

    print("\nTipos de Movimentação mais frequentes:")
    display(cluster_df['tipo_movimentacao'].value_counts().head())



# Iterate through each unique cluster and display its characteristics
for cluster_label in df_aggregated_movement['cluster'].unique():
    cluster_df = df_aggregated_movement[df_aggregated_movement['cluster'] == cluster_label]
    display_cluster_characteristics(cluster_df, cluster_label)

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy as np

# Select only the specified numeric columns
df_numeric_selected = df[['ano_movimentacao', 'qte_movimentacao']].copy()

# Ensure 'qte_movimentacao' is numeric, coercing errors
df_numeric_selected['qte_movimentacao'] = pd.to_numeric(df_numeric_selected['qte_movimentacao'], errors='coerce')

# Drop rows with NaN values that might have resulted from coercion
df_numeric_selected.dropna(subset=['qte_movimentacao'], inplace=True)

# Display dtypes before scaling to confirm
print("Data types of selected numeric columns before scaling:")
print(df_numeric_selected.dtypes)

# Scale the selected numeric data
scaler_selected = MinMaxScaler()
data_scaled_selected = scaler_selected.fit_transform(df_numeric_selected)

# Apply PCA with 2 components (since there are only 2 features)
pca_selected = PCA(n_components=2)
pca_components_selected = pca_selected.fit_transform(data_scaled_selected)

# Plot the explained variance ratio
plt.figure(figsize=(10, 6))
plt.plot(np.cumsum(pca_selected.explained_variance_ratio_))
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance Ratio')
plt.title('Explained Variance by PCA Components (Selected Features)')
plt.grid(True)
plt.show()

# Display the explained variance ratio per component
print("Explained Variance Ratio per Component (Selected Features):", pca_selected.explained_variance_ratio_)
print("Cumulative Explained Variance Ratio (Selected Features):", np.cumsum(pca_selected.explained_variance_ratio_))

In [ ]:
# Apply One-Hot Encoding to 'tipo_movimentacao'
df_encoded = pd.get_dummies(df, columns=['tipo_movimentacao'], drop_first=False)

# Display the first few rows with the new encoded columns
display(df_encoded.head())

In [ ]:
# Select the numeric columns and the one-hot encoded columns
features_for_pca = ['ano_movimentacao', 'qte_movimentacao', 'tipo_movimentacao_ENTRADA', 'tipo_movimentacao_SAIDA']
df_pca_features = df_encoded[features_for_pca].copy()

# Ensure 'qte_movimentacao' is numeric, coercing errors
df_pca_features['qte_movimentacao'] = pd.to_numeric(df_pca_features['qte_movimentacao'], errors='coerce')

# Drop rows with NaN values that might have resulted from coercion
df_pca_features.dropna(subset=['qte_movimentacao'], inplace=True)


# Display dtypes before scaling to confirm
print("Data types of features for PCA before scaling:")
print(df_pca_features.dtypes)


# Scale the selected features
scaler_pca = MinMaxScaler()
data_scaled_pca = scaler_pca.fit_transform(df_pca_features)

In [ ]:
# Apply PCA (choose number of components, e.g., 3 or based on explained variance)
# Let's start with 3 components as we have 4 features now
pca_full = PCA(n_components=3)
pca_components_full = pca_full.fit_transform(data_scaled_pca)

# Display the explained variance ratio
print("Explained Variance Ratio per Component (Full Features):", pca_full.explained_variance_ratio_)
print("Cumulative Explained Variance Ratio (Full Features):", np.cumsum(pca_full.explained_variance_ratio_))

# Get the principal components (eigenvectors)
pca_loadings_full_df = pd.DataFrame(pca_full.components_, columns=features_for_pca)

# Display the loadings
print("\nPCA Loadings (Full Features):")
display(pca_loadings_full_df)

# Add PCA components to a DataFrame for easier plotting, aligning with the original index
pca_df = pd.DataFrame(pca_components_full, index=df_pca_features.index, columns=['PC1', 'PC2', 'PC3'])

# Merge with the original DataFrame to get the 'ano_movimentacao' for coloring
# Ensure the indices are aligned. df_pca_features already aligns with df_encoded and df
pca_df_with_year = pca_df.join(df_encoded['ano_movimentacao'])

# Visualize the data in the PCA space, colored by 'ano_movimentacao'
plt.figure(figsize=(10, 6))
scatter = plt.scatter(pca_df_with_year['PC1'], pca_df_with_year['PC2'], c=pca_df_with_year['ano_movimentacao'], cmap='viridis', alpha=0.5) # Use alpha for better visibility
plt.title('Data Points in 2D PCA Space (PC1 vs PC2) colored by Year')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.colorbar(scatter, label='Year') # Add a color bar to show the mapping of colors to years
plt.grid(True)
plt.show()

In [ ]:
# Merge with the original DataFrame to get the 'tipo_movimentacao' for coloring
# Ensure the indices are aligned. pca_df aligns with df
pca_df_with_type = pca_df.join(df['tipo_movimentacao'])

plt.figure(figsize=(10, 6))
# Use the 'tipo_movimentacao' column for coloring
scatter = plt.scatter(pca_df_with_type['PC1'], pca_df_with_type['PC2'], c=pca_df_with_type['tipo_movimentacao'].astype('category').cat.codes, cmap='viridis', alpha=0.5) # Convert to category codes for coloring
plt.title('Data Points in 2D PCA Space (PC1 vs PC2) colored by Movement Type')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
# Add a color bar with labels for movement types
cbar = plt.colorbar(scatter)
cbar.set_ticks(range(len(pca_df_with_type['tipo_movimentacao'].unique())))
cbar.set_ticklabels(pca_df_with_type['tipo_movimentacao'].unique())
cbar.set_label('Movement Type')

plt.grid(True)
plt.show()

In [ ]:
from sklearn.metrics import davies_bouldin_score
from sklearn.cluster import KMeans

# Assuming you have already determined the optimal number of clusters (e.g., 6 from the elbow method)
# and have your scaled aggregated data in 'data_aggregated_scaled' and your KMeans model in 'kmeans'

# Get the cluster labels from the fitted KMeans model
cluster_labels = kmeans.labels_

# Calculate the Davies-Bouldin Index
db_score_aggregated = davies_bouldin_score(data_aggregated_scaled, cluster_labels)

print(f"Davies-Bouldin Index for the aggregated data with 6 clusters: {db_score_aggregated}")

# Product Movement Pattern Analysis - In-Depth Methodology for Data Scientists/ML Engineers

This document provides a comprehensive technical breakdown of the methodology employed in this project, tailored for a Data Scientist or ML Engineer audience. It delves into the rationale behind each step and the selection of specific techniques, emphasizing the connection to the core business rule of identifying areas of significant product movement.

## 1. Data Ingestion and Initial Assessment

The process began with ingesting the product movement dataset. A critical initial assessment was performed to understand the data's schema, volume, and potential data quality issues. This involved:

*   **Schema Inspection:** Examining column names, data types, and their potential relevance to product movement analysis (e.g., product category, stock location name, movement quantity, movement type).
*   **Volume Assessment:** Understanding the number of records to gauge the computational resources required for subsequent processing.
*   **Initial Quality Checks:** Quick scans for obvious data anomalies, such as unexpected data types, out-of-range values, or inconsistent formatting.

The objective here was to establish a solid understanding of the raw data's landscape before any transformation.

## 2. Data Wrangling and Feature Engineering

This stage focused on transforming the raw data into a format suitable for analysis and feature engineering to capture relevant aspects of product movement:

*   **Handling Erroneous Records:** Specifically addressing and removing records identified with improbable values, such as movement years with unlikely values (e.g., 2302), as these would distort temporal analysis and downstream modeling. The decision to remove rather than impute or correct was based on the lack of clear business context for such outliers and their potential to negatively impact model performance.
*   **String Normalization:** Applying string cleaning operations (like removing leading/trailing whitespace) to text columns was essential to eliminate inconsistencies. This seemingly minor step is crucial for preventing issues in grouping, filtering, and categorical encoding due to variations in spacing.
*   **Handling Missing Values:** While the initial assessment showed no nulls, the potential for empty strings after stripping was considered. Checks for empty or whitespace-only cells provided a robust check for these cases, indicating that no empty string imputation or handling was needed for the relevant columns.
*   **Data Type Conversion:** Ensuring that the movement quantity was treated as a numeric type (`float`) was vital for aggregation and quantitative analysis. Using a robust conversion method with error coercion allowed for graceful handling of any non-numeric entries by converting them to `NaN`, which were subsequently dropped where necessary for specific analyses requiring clean numerical data.

## 3. Feature Selection and Aggregation for Business Insight

Drawing a direct line to the business rule of identifying areas of high movement, key features were selected to define a "movement event":

*   Product category, product subcategory, stock location name, branch: These dimensions define *where* and *what* was moved, providing the contextual basis for identifying significant areas.
*   Movement year, movement type: These dimensions capture the *when* and *how* of the movement, adding temporal and directional context.
*   Movement quantity: This is the core metric, quantifying the *magnitude* of the movement.

The aggregation step (grouping by the selected dimensions and summing the movement quantity) was a deliberate choice to move from granular transaction data to a summary of total movement quantity for each unique combination of the selected dimensions. This aggregated dataset became the foundation for identifying distinct movement patterns, as it encapsulates the total flow for specific product types at specific locations and times, categorized by movement type.

## 4. Feature Scaling for Distance-Based Algorithms

For algorithms sensitive to the magnitude of features, such as K-Means clustering and PCA, feature scaling is a mandatory preprocessing step. The **MinMaxScaler** was chosen because it transforms features to a specific range (typically 0 to 1) without distorting the relationships between values. This is appropriate when the distribution of the original data is not necessarily Gaussian and the goal is to ensure all features contribute proportionally to distance calculations, regardless of their original scale. The scaling was applied to the numeric features relevant for clustering and PCA.

## 5. Unsupervised Learning: K-Means Clustering

To identify inherent groupings or "patterns" within the aggregated movement data, **K-Means clustering** was employed. The rationale for using K-Means is its simplicity, efficiency, and interpretability. It partitions the data into a pre-defined number of clusters based on minimizing the within-cluster sum of squares. This aligns perfectly with the goal of grouping similar movement events together based on their aggregated quantity and categorical attributes (implicitly captured through the aggregation structure and potentially through one-hot encoding for PCA).

*   **Determining K (Elbow Method):** The Elbow Method is a heuristic used to estimate the optimal number of clusters. By plotting the inertia (a measure of how well the data is clustered) against the number of clusters, the "elbow point" indicates a good balance between the complexity of the model and the reduction in within-cluster variance. This data-driven approach avoided arbitrary selection of the number of clusters.
*   **Cluster Analysis:** Post-clustering, a critical step was analyzing the characteristics of each resulting cluster. This involved examining the descriptive statistics of movement quantity within each cluster and the frequency distribution of the categorical features (product category, branch, etc.). This step directly links the abstract clusters back to the business context, allowing for the identification of patterns like "high-volume movements of a specific Product Type at a specific Location" or "low-volume movements of another Product Type at a different Location". A utility function was developed to automate this analysis for each cluster, facilitating rapid interpretation.

## 6. Clustering Validation: Davies-Bouldin Index

The **Davies-Bouldin Index (DBI)** was calculated to provide a quantitative measure of the quality of the clustering solution. The DBI is an internal validation metric that evaluates the compactness and separation of the clusters. A lower DBI value indicates better clustering, with clusters being more internally cohesive and externally distinct. The obtained very low DBI score strongly suggests that the K-Means algorithm, with the chosen number of clusters, produced a robust and meaningful partitioning of the aggregated movement data. This metric provides objective evidence supporting the validity of the identified movement patterns.

## 7. Dimensionality Reduction and Feature Contribution: Principal Component Analysis (PCA)

**Principal Component Analysis (PCA)** was used for two primary reasons: to reduce the dimensionality of the data while retaining most of the variance, and to understand the contribution of the original features to the principal components.

*   **Rationale for PCA:** Even with a relatively small number of features after aggregation, PCA helps visualize the data in a lower-dimensional space (2D or 3D plots) while capturing the most significant patterns of variation. It also helps in understanding which features are most important in differentiating the data points.
*   **Feature Selection for PCA:** Initially, PCA was applied to just the numeric features (movement year, movement quantity) to see their individual contribution to variance. Subsequently, **One-Hot Encoding** was applied to the movement type categorical feature to include its influence in the PCA. One-Hot Encoding is a standard technique for representing categorical variables as numerical vectors, making them suitable for algorithms like PCA that operate on numerical data. The decision to include movement type in a separate PCA run was to explicitly investigate how movement direction (entry/exit) contributes to the overall data variance alongside year and quantity.
*   **Interpreting Explained Variance and Loadings:** Examining the explained variance ratio per component revealed how much of the total data variability was captured by each principal component. The cumulative explained variance indicated how many components were needed to explain a significant portion of the variance. The PCA loadings (eigenvectors) showed the weight of each original feature in the principal components. High absolute loading values indicate that a feature has a strong influence on that component. For instance, the loadings showed that movement year and movement type were major contributors to the principal components, confirming their importance in shaping the primary dimensions of variability in the data.
*   **Visualization in PCA Space:** Plotting the data points in the 2D PCA space (PC1 vs PC2), colored by movement year and movement type, provided intuitive visualizations of how these features drive the separation and grouping of data points in the reduced-dimensional space. The clustering of points by year and the clear separation based on movement type in the PCA plots visually corroborated the insights gained from the cluster analysis and reinforced the importance of these features in defining distinct movement patterns.

## Conclusion from a Data Science Perspective

This project successfully leveraged a standard machine learning pipeline (preprocessing, feature engineering, scaling, clustering, validation, and dimensionality reduction) to extract meaningful patterns from sensitive product movement data. The selection of K-Means was justified by the need for interpretable clusters tied to business dimensions. The use of the Elbow Method and Davies-Bouldin Index provided rigorous validation for the chosen clustering solution. PCA not only aided in visualization but also provided crucial insights into the feature importance, confirming the business intuition about the significance of year and movement type. The identified clusters represent actionable segments of product movement, providing a data-driven foundation for optimizing inventory management, logistics, and strategic planning, all while respecting data confidentiality by focusing on the methodology and derived patterns rather than the raw data itself.